# University of Michigan
## School of Information
**Masters of Applied Data Science**<br>
**Milestone 1**<br>
<br>
Affordable Housing Project
<br><br>
**Submitted by**<br>
>Alan Fehsenfeld<br>Koon Leong Ho<br>George Thio

# Append Amenities Indicator to Municipality File Records

## Import Modules

In [7]:
from math import radians, cos, sin, asin, sqrt
import pandas as pd
import geopandas
from config import basedir

base = basedir()


"/Users/koonleong/Documents/UMMADS/SIADS591/project_git/go_blue/src/final/data/municipalities/All Municipalities Geometry Only.shp'"

## Append Amenity Proximity to Municipality File

In [36]:
def amenity_proximity(csv):
    """take csv file name as input, read amenities csv and All Municipalities Geometry Only csv, calculate distance from AMGO file data points
    to all amenities and determine if AMGO file data points have amenity within one mile. Output dataframe with
    Original APN and amenities within one mile indicator."""
    
    # Setup the full name to the datafile
    input_file = r"/data/municipalities/All Municipalities Geometry Only.shp"
    fp_input_file = base + input_file

#     The following line of code should be deleted after the revised read_file has been tested to be correct
#     AMGO=geopandas.read_file('documents/UofM/SIADS 591/All Municipalities Geometry Only.shp')

    AMGO=geopandas.read_file(fp_input_file)
    AMGO_geo=AMGO.buffer(5280)
    AMGO.geometry=AMGO_geo
    
    amenities=pd.read_csv(csv)
    amenities=geopandas.GeoDataFrame(amenities,geometry=geopandas.points_from_xy(amenities.loc[:,'Long'],amenities.loc[:,'Lat']))
    amenities=amenities.set_crs('WGS84')
    amenities=amenities.to_crs('EPSG:2253')
    
    amenities=geopandas.sjoin(amenities,AMGO,how='right',op='intersects')
    amenities=amenities.loc[:,['ID','index_left']]
    amenities=amenities.rename(columns={'index_left':'amenities_ind'})
    
    amenities['amenities_ind']=amenities['amenities_ind'].where(amenities['amenities_ind'].isna(),1)
    amenities['amenities_ind']=amenities['amenities_ind'].where(amenities['amenities_ind'].notna(),0)

    amenities=amenities.groupby(['ID']).agg('max').reset_index()
    
    return(amenities)

## Import Each Amenity File and Run the Amenity_Proximity Function on Them


In [40]:
input_file = r"/data/raw/groceries.csv"
output_file = r"/data/groceries_ind.csv"

fp_input_file = base + input_file
fp_output_file = base + output_file

# to be deleted after testing
# groceries_df=amenity_proximity('documents/UofM/SIADS 591/groceries.csv')
# groceries_df.to_csv('documents/UofM/SIADS 591/groceries_ind.csv',index=False)

groceries_df=amenity_proximity(fp_input_file)
groceries_df.to_csv(fp_output_file,index=False)

In [41]:
input_file = r"/data/raw/pharmacies.csv"
output_file = r"/data/pharmacies_ind.csv"

fp_input_file = base + input_file
fp_output_file = base + output_file

# to be deleted after testing
# pharmacies_df=amenity_proximity('documents/UofM/SIADS 591/pharmacies.csv')
# pharmacies_df.to_csv('documents/UofM/SIADS 591/pharmacies_ind.csv',index=False)

pharmacies_df=amenity_proximity(fp_input_file)
pharmacies_df.to_csv(fp_output_file,index=False)

In [42]:
input_file = r"/data/raw/doctors.csv"
output_file = r"/data/doctors_ind.csv"

fp_input_file = base + input_file
fp_output_file = base + output_file

# to be deleted after testing
# doctors_df=amenity_proximity('documents/UofM/SIADS 591/doctors.csv')
# doctors_df.to_csv('documents/UofM/SIADS 591/doctors_ind.csv',index=False)

doctors_df=amenity_proximity(fp_input_file)
doctors_df.to_csv(fp_output_file,index=False)

In [43]:
input_file = r"/data/raw/parks.csv"
output_file = r"/data/parks_ind.csv"

fp_input_file = base + input_file
fp_output_file = base + output_file

# to be deleted after testing
# parks_df=amenity_proximity('documents/UofM/SIADS 591/parks.csv')
# parks_df.to_csv('documents/UofM/SIADS 591/parks_ind.csv',index=False)

parks_df=amenity_proximity('documents/UofM/SIADS 591/parks.csv')
parks_df.to_csv('documents/UofM/SIADS 591/parks_ind.csv',index=False)

In [44]:
input_file = r"/data/raw/schools.csv"
output_file = r"/data/schools_ind.csv"

fp_input_file = base + input_file
fp_output_file = base + output_file


# schools_df=amenity_proximity('documents/UofM/SIADS 591/schools.csv')
# schools_df.to_csv('documents/UofM/SIADS 591/schools_ind.csv',index=False)

schools_df=amenity_proximity(fp_input_file)
schools_df.to_csv(fp_output_file,index=False)


## Put Together the Amenity Files and Calculate MSHDA Score

In [46]:
groceries_ind=groceries_df.loc[:,'amenities_ind']
pharmacies_ind=pharmacies_df.loc[:,'amenities_ind']
doctors_ind=doctors_df.loc[:,'amenities_ind']
parks_ind=parks_df.loc[:,'amenities_ind']
schools_ind=schools_df.loc[:,'amenities_ind']

amenities=groceries_ind+pharmacies_ind+doctors_ind+parks_ind+schools_ind

amenities_df=pd.DataFrame({'ID':groceries_df.loc[:,'ID'],'amenities_ind':amenities})

In [48]:
amenities=pd.DataFrame(amenities_df.loc[:,'amenities_ind'])
amenities=amenities.where(amenities!=5,4)
amenities_df.loc[:,'amenities_ind']=amenities


In [ ]:
output_file = r"/data/schools_ind.csv"
fp_output_file = base + output_file

# to be deleted after testing
# amenities_df.to_csv('documents/UofM/SIADS 591/Amenities.csv',index=False)

amenities_df.to_csv(fp_output_file,index=False)